In [81]:
include("framework.jl")
include("create.jl")
configPath="../data/config";

In [82]:
config = initConfig();
seed = 0;  Random.seed!(seed);

#cubicArray( dimension, size, config, theta, resolution, radius1, radius2, radius3, fAdd=false )
cubicArray!( 5, 3, config, 0, 3, 0.5, 0.5, 0.5, true )
cubicArray!( 3, 6, config, 1, 3, 0.5, 0.5, 0.5, true )
write_csv(config, configPath*"/test.csv")

"../data/config/test.csv"